In [2]:
import json
from llama_cpp import Llama
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [3]:
system_prompt = """
Given the sentence, generate as many paraphrased sentences as possible while preserving the original semantic meaning and style. 
Return the rephrased sentences in a JSON format. Try to return at least 20 different paraphrases in the output.
DO NOT INCLUDE ANY NOTES OR ADDITIONAL TEXT IN THE OUTPUT.
"""

examples = [
    {
        "original_user_supplied_sentence": "Known for being very delicate, the skill could take a lifetime to master.",
        "new_assistant_rephrased_sentence": [
            "The skill is well known for its delicacy and could require a lifetime to perfect.",
            "The skill's reputation for delicateness suggests that it could take a whole lifetime to master.",
            "It may take a lifetime to master the skill, which is renowned for its delicacy.",
            "The delicacy of the skill means it could take a lifetime to master."
        ]
    }
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{original_user_supplied_sentence}"),
        ("ai", "{new_assistant_rephrased_sentence}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        few_shot_prompt,
        ("human", "{original_user_supplied_sentence}"),
    ]
)

In [4]:
def convert_to_phi(original_sentence,
                   prompt_input=final_prompt):

    messages = prompt_input.messages
    
    formatted_messages = ""

    for message in messages:
        if isinstance(message, SystemMessagePromptTemplate):
            formatted_messages += f"<|assistant|>\n{message.prompt.template.replace('\n', '')} <|end|>\n"
        elif isinstance(message, FewShotChatMessagePromptTemplate):
            formatted_messages += f"<|user|>\n{message.examples[0]['original_user_supplied_sentence'].replace('\n', '')} <|end|>\n"
            formatted_messages += f"<|assistant|>\n{message.examples[0]} <|end|>\n"
        elif isinstance(message, HumanMessagePromptTemplate):
            formatted_messages += f"<|user|>\n{message.prompt.template.replace('\n', '')} <|end|>\n"
    
    formatted_messages += f"<|assistant|>"

    formatted_prompt = formatted_messages.replace("<|user|>\n{original_user_supplied_sentence} <|end|>", f"<|user|>\n{original_sentence} <|end|>")
    
    return formatted_prompt
    

In [5]:
llm = Llama(
    model_path="C:/Users/benjc/Documents/models/Phi-3-mini-4k-instruct-q4.gguf",
    n_ctx=4096,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=10, # Maximum I have is 12
    n_gpu_layers=-1, # The number of layers to offload to GPU, if you have GPU acceleration available.
    verbose=False,
    flash_attn=True
)

In [ ]:
def phi_paraphrase(original_sentence, n_runs,
                   llm=llm,
                   prompt_input=final_prompt):


    formatted_prompt = convert_to_phi(original_sentence, prompt_input=final_prompt)

    results = []
    
    for i in range(1, n_runs + 1):
        
        print(f'  Iteration: {i}')
        attempts = 1
        
        while True:
            try:
                output_str = llm(formatted_prompt, max_tokens=1000, stop=["<|end|>"])
                output_text = output_str['choices'][0]['text']

                output_key_format = output_text.replace("'original_user_supplied_sentence':", '"sentence":')
                output_key_format = output_key_format.replace('"original_user_supplied_sentence":', '"sentence":')
                output_key_format = output_key_format.replace("'new_assistant_rephrased_sentences':", '"rephrased":')
                output_key_format = output_key_format.replace('"new_assistant_rephrased_sentences":', '"rephrased":')

                json_response = json.loads(output_key_format)
                break
            except Exception as e:
                attempts += 1  # Increment the number of attempts
                print(f'    Attempt {attempts} failed: {str(e)}')
                
        print(f'    Attempts: {attempts}')
        results.append(json_response)
        
    return results

In [ ]:
results = phi_paraphrase("DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.", n_runs=10)

  Iteration: 1
    Attempt 1 failed: Invalid control character at: line 6 column 108 (char 537)
    Attempt 2 failed: Expecting ',' delimiter: line 22 column 113 (char 1171)
    Attempts: 2
  Iteration: 2
    Attempt 1 failed: Expecting value: line 2 column 14 (char 14)
    Attempts: 1
  Iteration: 3
    Attempts: 0
  Iteration: 4
    Attempts: 0
  Iteration: 5
    Attempt 1 failed: Expecting value: line 2 column 14 (char 14)
    Attempt 2 failed: Expecting value: line 2 column 14 (char 14)
    Attempt 3 failed: Expecting value: line 2 column 14 (char 14)
    Attempts: 3
  Iteration: 6
    Attempt 1 failed: Expecting value: line 2 column 14 (char 14)
    Attempt 2 failed: Expecting value: line 2 column 118 (char 118)
    Attempt 3 failed: Expecting value: line 2 column 118 (char 118)
    Attempts: 3
  Iteration: 7
    Attempt 1 failed: Expecting value: line 2 column 14 (char 14)
    Attempt 2 failed: Expecting value: line 1 column 15 (char 14)
    Attempt 3 failed: Expecting value: lin

In [64]:
results

[' {\n    "original_user_supplied_sentence": "DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.",\n    "new_assistant_rephrased_sentences": [\n        "The esteemed Booker Prize recipient DBC Pierre, acclaimed for writing \'Vernon God Little\', has relocated to Ireland.", \n        "Ireland is now home to the celebrated author and former Booker Prize laureate DBC Pierre, who penned Vernon God Little.",\n        "Author of \'Vernon God Little\' and a past winner of the Booker Prize, DBC Pierre has recently moved to Ireland.", \n        "\'Vernon God Little\'s writer, the distinguished Booker Prize honoree DBC Pierre, now resides in Ireland.", \n        "DBC Pierre, recognized for his work on \'Vernon God Little\' and a recipient of the Booker Prize, has recently made Ireland his new residence.",\n        "The acclaimed author known for writing Vernon God Little, along with being a previous winner of the Booker Prize, is DBC Pierre, who now lives in I

In [81]:
def encode_quotes(json_string):
    encoded_quotes = []
    encoded_json_string = json_string

    while True:
        # Find the next single quote
        quote_pos = encoded_json_string.find("'")
        if quote_pos == -1:
            break

        # Encode the single quote and save its position
        encoded_quotes.append(quote_pos)
        encoded_json_string = encoded_json_string[:quote_pos] + "§" + encoded_json_string[quote_pos + 1:]

    return encoded_json_string, encoded_quotes

def fix_json(json_string, encoded_quotes):
    # Replace single quotes with double quotes
    json_string = json_string.replace("'", '"')

    # Restore the single quotes from their encoded positions
    for pos in encoded_quotes:
        json_string = json_string[:pos] + "'" + json_string[pos + 1:]

    return json_string

In [89]:
final_json = []
for result in results:

    result_key_format = result.replace("'original_user_supplied_sentence':", '"sentence":')
    result_key_format = result_key_format.replace("""original_user_supplied_sentence:""", '"sentence":')
    result_key_format = result_key_format.replace("'new_assistant_rephrased_sentences':", '"rephrased":')
    result_key_format = result_key_format.replace("""new_assistant_rephrased_sentences:""", '"rephrased":')

    result_key_string, result_key_encoded = encode_quotes(result_key_format)
    print(result_key_string)
    
    result_key_fixed = fix_json(result_key_string, result_key_encoded)
    print(result_key_fixed)

    result_key_json = json.loads(result_key_fixed)
    
    final_json.append(result_key_json)

    

 {
    "original_user_supplied_sentence": "DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.",
    "new_assistant_rephrased_sentences": [
        "The esteemed Booker Prize recipient DBC Pierre, acclaimed for writing §Vernon God Little§, has relocated to Ireland.", 
        "Ireland is now home to the celebrated author and former Booker Prize laureate DBC Pierre, who penned Vernon God Little.",
        "Author of §Vernon God Little§ and a past winner of the Booker Prize, DBC Pierre has recently moved to Ireland.", 
        "§Vernon God Little§s writer, the distinguished Booker Prize honoree DBC Pierre, now resides in Ireland.", 
        "DBC Pierre, recognized for his work on §Vernon God Little§ and a recipient of the Booker Prize, has recently made Ireland his new residence.",
        "The acclaimed author known for writing Vernon God Little, along with being a previous winner of the Booker Prize, is DBC Pierre, who now lives in Ireland.", 
       

JSONDecodeError: Expecting value: line 1 column 119 (char 118)

In [ ]:
except Exception as e:
                attempts += 1  # Increment the number of attempts
                print(f'    Attempt {attempts} failed: {str(e)}')

In [86]:
final_json

[' {\n    "original_user_supplied_sentence": "DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.",\n    "new_assistant_rephrased_sentences": [\n        "The esteemed Booker Prize recipient DBC Pierre, acclaimed for writing \'Vernon God Little\', has relocated to Ireland.", \n        "Ireland is now home to the celebrated author and former Booker Prize laureate DBC Pierre, who penned Vernon God Little.",\n        "Author of \'Vernon God Little\' and a past winner of the Booker Prize, DBC Pierre has recently moved to Ireland.", \n        "\'Vernon God Little\'s writer, the distinguished Booker Prize honoree DBC Pierre, now resides in Ireland.", \n        "DBC Pierre, recognized for his work on \'Vernon God Little\' and a recipient of the Booker Prize, has recently made Ireland his new residence.",\n        "The acclaimed author known for writing Vernon God Little, along with being a previous winner of the Booker Prize, is DBC Pierre, who now lives in I

In [65]:
test = results[0].replace("'original_user_supplied_sentence':", '"sentence":')
test = test.replace("'new_assistant_rephrased_sentences':", '"rephrased":')
test = test.strip()

In [66]:
print(test)

{
    "original_user_supplied_sentence": "DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.",
    "new_assistant_rephrased_sentences": [
        "The esteemed Booker Prize recipient DBC Pierre, acclaimed for writing 'Vernon God Little', has relocated to Ireland.", 
        "Ireland is now home to the celebrated author and former Booker Prize laureate DBC Pierre, who penned Vernon God Little.",
        "Author of 'Vernon God Little' and a past winner of the Booker Prize, DBC Pierre has recently moved to Ireland.", 
        "'Vernon God Little's writer, the distinguished Booker Prize honoree DBC Pierre, now resides in Ireland.", 
        "DBC Pierre, recognized for his work on 'Vernon God Little' and a recipient of the Booker Prize, has recently made Ireland his new residence.",
        "The acclaimed author known for writing Vernon God Little, along with being a previous winner of the Booker Prize, is DBC Pierre, who now lives in Ireland.", 
        

In [67]:
json.loads(test)

{'original_user_supplied_sentence': 'DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.',
 'new_assistant_rephrased_sentences': ["The esteemed Booker Prize recipient DBC Pierre, acclaimed for writing 'Vernon God Little', has relocated to Ireland.",
  'Ireland is now home to the celebrated author and former Booker Prize laureate DBC Pierre, who penned Vernon God Little.',
  "Author of 'Vernon God Little' and a past winner of the Booker Prize, DBC Pierre has recently moved to Ireland.",
  "'Vernon God Little's writer, the distinguished Booker Prize honoree DBC Pierre, now resides in Ireland.",
  "DBC Pierre, recognized for his work on 'Vernon God Little' and a recipient of the Booker Prize, has recently made Ireland his new residence.",
  'The acclaimed author known for writing Vernon God Little, along with being a previous winner of the Booker Prize, is DBC Pierre, who now lives in Ireland.',
  "DBC Pierre, an award-winning writer of 'Vernon God Littl

In [33]:
system_prompt = """
Given the sentence, generate as many paraphrased sentences as possible while preserving the original semantic meaning and style. 
Return the rephrased sentences in a python list format. Try to return at least 20 different paraphrases in the output.
DO NOT INCLUDE ANY NOTES OR ADDITIONAL TEXT IN THE OUTPUT.

An example is below:
--------
Sentence: ```"Known for being very delicate, the skill could take a lifetime to master."```

Rephrased Sentences: ```["The skill is well known for its delicacy and could require a lifetime to perfect.", "The skill's reputation for delicateness suggests that it could take a whole lifetime to master.", "It may take a lifetime to master the skill, which is renowned for its delicacy.", "The delicacy of the skill means it could take a lifetime to master."]```
--------
Sentence: ```{original_user_supplied_sentence}```
"""

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{original_user_supplied_sentence}"),
    ]
)

In [34]:
formatted_prompt = convert_to_phi(original_sentence, prompt_input=final_prompt)

In [35]:
formatted_prompt

'<|assistant|>\nGiven the sentence, generate as many paraphrased sentences as possible while preserving the original semantic meaning and style. Return the rephrased sentences in a python list format. Try to return at least 20 different paraphrases in the output.DO NOT INCLUDE ANY NOTES OR ADDITIONAL TEXT IN THE OUTPUT.An example is below:--------Sentence: ```"Known for being very delicate, the skill could take a lifetime to master."```Rephrased Sentences: ```["The skill is well known for its delicacy and could require a lifetime to perfect.", "The skill\'s reputation for delicateness suggests that it could take a whole lifetime to master.", "It may take a lifetime to master the skill, which is renowned for its delicacy.", "The delicacy of the skill means it could take a lifetime to master."]```--------Sentence: ```{original_user_supplied_sentence}``` <|end|>\n<|user|>\nDBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland. <|end|>\n<|assistant|>'

In [36]:
original_sentence = "DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland."
formatted_prompt = convert_to_phi(original_sentence, prompt_input=final_prompt)
print(formatted_prompt)

<|assistant|>
Given the sentence, generate as many paraphrased sentences as possible while preserving the original semantic meaning and style. Return the rephrased sentences in a python list format. Try to return at least 20 different paraphrases in the output.DO NOT INCLUDE ANY NOTES OR ADDITIONAL TEXT IN THE OUTPUT.An example is below:--------Sentence: ```"Known for being very delicate, the skill could take a lifetime to master."```Rephrased Sentences: ```["The skill is well known for its delicacy and could require a lifetime to perfect.", "The skill's reputation for delicateness suggests that it could take a whole lifetime to master.", "It may take a lifetime to master the skill, which is renowned for its delicacy.", "The delicacy of the skill means it could take a lifetime to master."]```--------Sentence: ```{original_user_supplied_sentence}``` <|end|>
<|user|>
DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland. <|end|>
<|assistant|>


In [37]:
result = llm(formatted_prompt, max_tokens=1000, stop=["<|end|>"])

In [38]:
result_str = result['choices'][0]['text']

In [39]:
result_str

' ["Pierre DBC, the recipient of the prestigious Booker Prize and writer of \'Vernon God Little,\' recently relocated to Ireland.", "The acclaimed author Pierre DBC, who penned \'Vernon God Little\' and won the Booker Prize, has now made Ireland his home.", "\'Vernon God Little\' author and Booker Prize laureate Pierre DBC has decided to settle in Ireland.", "Pierre DBC, celebrated for \'Vernon God Little\' and receiving the Booker Prize, has recently embarked on a new life in Ireland.", "As an esteemed writer of \'Vernon God Little,\' recipient of the Booker Prize, Pierre DBC is now residing in Ireland.", "\'Vernon God Little\' creator and Booker Prize winner Pierre DBC has relocated to Ireland.", "Author of \'Vernon God Little\' and a proud holder of the Booker Prize title, Pierre DBC has now called Ireland his residence.", "With the accolade of winning the Booker Prize for \'Vernon God Little,\' Pierre DBC is currently residing in Ireland.", "\'Vernon God Little\' author and recipie

In [52]:
# Find the index of the first '[' and the last ']'
start_index = result_str.find('[')
end_index = result_str.rfind(']')

# Extract the content between the first '[' and the last ']'
content_str = result_str[start_index+1:end_index]

# Evaluate the content string as a Python expression to convert it into a list
result_list = eval('[' + content_str + ']')


In [53]:
result_list

["Pierre DBC, the recipient of the prestigious Booker Prize and writer of 'Vernon God Little,' recently relocated to Ireland.",
 "The acclaimed author Pierre DBC, who penned 'Vernon God Little' and won the Booker Prize, has now made Ireland his home.",
 "'Vernon God Little' author and Booker Prize laureate Pierre DBC has decided to settle in Ireland.",
 "Pierre DBC, celebrated for 'Vernon God Little' and receiving the Booker Prize, has recently embarked on a new life in Ireland.",
 "As an esteemed writer of 'Vernon God Little,' recipient of the Booker Prize, Pierre DBC is now residing in Ireland.",
 "'Vernon God Little' creator and Booker Prize winner Pierre DBC has relocated to Ireland.",
 "Author of 'Vernon God Little' and a proud holder of the Booker Prize title, Pierre DBC has now called Ireland his residence.",
 "With the accolade of winning the Booker Prize for 'Vernon God Little,' Pierre DBC is currently residing in Ireland.",
 "'Vernon God Little' author and recipient of the Bo

In [50]:
# Split the content string into individual lines
lines = content_str.split('\n')
lines

['["Pierre DBC, the recipient of the prestigious Booker Prize and writer of \'Vernon God Little,\' recently relocated to Ireland.", "The acclaimed author Pierre DBC, who penned \'Vernon God Little\' and won the Booker Prize, has now made Ireland his home.", "\'Vernon God Little\' author and Booker Prize laureate Pierre DBC has decided to settle in Ireland.", "Pierre DBC, celebrated for \'Vernon God Little\' and receiving the Booker Prize, has recently embarked on a new life in Ireland.", "As an esteemed writer of \'Vernon God Little,\' recipient of the Booker Prize, Pierre DBC is now residing in Ireland.", "\'Vernon God Little\' creator and Booker Prize winner Pierre DBC has relocated to Ireland.", "Author of \'Vernon God Little\' and a proud holder of the Booker Prize title, Pierre DBC has now called Ireland his residence.", "With the accolade of winning the Booker Prize for \'Vernon God Little,\' Pierre DBC is currently residing in Ireland.", "\'Vernon God Little\' author and recipie

In [43]:
# Split the content string into individual lines
lines = content_str.split('\n    ')

# Remove empty lines and leading/trailing whitespaces
lines = [line.strip() for line in lines if line.strip()]

In [45]:
result_list = []
for line in lines:
    # Remove the leading and trailing double quotes and add to the list
    result_list.append(line.strip()[1:-1])

result_list

['Pierre DBC, the recipient of the prestigious Booker Prize and writer of \'Vernon God Little,\' recently relocated to Ireland.", "The acclaimed author Pierre DBC, who penned \'Vernon God Little\' and won the Booker Prize, has now made Ireland his home.", "\'Vernon God Little\' author and Booker Prize laureate Pierre DBC has decided to settle in Ireland.", "Pierre DBC, celebrated for \'Vernon God Little\' and receiving the Booker Prize, has recently embarked on a new life in Ireland.", "As an esteemed writer of \'Vernon God Little,\' recipient of the Booker Prize, Pierre DBC is now residing in Ireland.", "\'Vernon God Little\' creator and Booker Prize winner Pierre DBC has relocated to Ireland.", "Author of \'Vernon God Little\' and a proud holder of the Booker Prize title, Pierre DBC has now called Ireland his residence.", "With the accolade of winning the Booker Prize for \'Vernon God Little,\' Pierre DBC is currently residing in Ireland.", "\'Vernon God Little\' author and recipient

In [72]:
def phi_paraphrase(original_sentence, n_runs,
                   llm=llm,
                   prompt_input=final_prompt):


    formatted_prompt = convert_to_phi(original_sentence, prompt_input=final_prompt)

    sentences = [original_sentence]
    new_sentence_amount = 1
    
    for i in range(1, n_runs + 1):

        if new_sentence_amount == 0:
            break
        print(f'  Iteration: {i}')
        attempts = 1
        
        while True:
            try:
                output_str = llm(formatted_prompt, max_tokens=1000, stop=["<|end|>"],
                                temperature=1)
                output_text = output_str['choices'][0]['text']

                # Find the index of the first '[' and the last ']'
                start_index = output_text.find('[')
                end_index = output_text.rfind(']')

                # Extract the content between the first '[' and the last ']'
                content_str = output_text[start_index+1:end_index]

                # Evaluate the content string as a Python expression to convert it into a list
                result_list = eval('[' + content_str + ']')
        
                break
            except Exception as e:
                attempts += 1  # Increment the number of attempts
                print(f'    Attempt {attempts} failed: {str(e)}')
                
        print(f'    Attempts: {attempts}')

        new_sentence_amount = 0

        for result in result_list:
            if result not in sentences:
                sentences.append(result)
                new_sentence_amount += 1
        
    return sentences

In [73]:
results = phi_paraphrase("DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.", n_runs=20)

  Iteration: 1
    Attempts: 1
  Iteration: 2
    Attempts: 1
  Iteration: 3
    Attempts: 1
  Iteration: 4
    Attempts: 1
  Iteration: 5
    Attempts: 1
  Iteration: 6
    Attempts: 1
  Iteration: 7
    Attempts: 1
  Iteration: 8
    Attempts: 1
  Iteration: 9
    Attempts: 1
  Iteration: 10
    Attempts: 1
  Iteration: 11
    Attempts: 1
  Iteration: 12
    Attempts: 1
  Iteration: 13
    Attempt 2 failed: invalid syntax (<string>, line 2)
    Attempts: 2
  Iteration: 14
    Attempts: 1
  Iteration: 15
    Attempts: 1
  Iteration: 16
    Attempts: 1
  Iteration: 17
    Attempts: 1
  Iteration: 18
    Attempts: 1
  Iteration: 19
    Attempts: 1
  Iteration: 20
    Attempt 2 failed: unterminated string literal (detected at line 21) (<string>, line 21)
    Attempts: 2


In [74]:
results

['DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.',
 "Pierre DBC, the acclaimed writer who won the Booker Prize for his work 'Vernon God Little,' has relocated to Ireland.",
 "The celebrated Booker Prize laureate, Pierre DBC, now calls Ireland home after completing 'Vernon God Little.'",
 'Author of the critically-acclaimed book Vernon God Little and a recipient of the prestigious Booker Prize, Pierre DBC has recently moved to Ireland.',
 'Irish landscape now features author Pierre DBC - the Booker Prize winner for his novel Vernon God Little.',
 "Famed Booker Prize-winning writer Pierre DBC, creator of 'Vernon God Little,' has made Ireland his new residence.",
 'Pierre DBC, esteemed author and Booker Prize victor with his book Vernon God Little, has now settled in Ireland.',
 "Having triumphed at the Booker Prize with his work 'Vernon God Little,' Pierre DBC - a renowned French novelist - is currently residing in Ireland.",
 "A laureate of the co

In [75]:
len(results)

265

In [59]:
# Define the chat GPT model
def gpt(prompt_input):

    messages = [
        {"role": "system", "content": prompt_input.messages[0].content},
        {"role": "user", "content": prompt_input.messages[1].content},
        {"role": "assistant", "content": prompt_input.messages[2].content},
        {"role": "user", "content": prompt_input.messages[-1].content}
    ]

    response = llm.create_chat_completion(
        messages=messages,
        temperature=0.7,
        max_tokens=4000
    )
    return response

In [60]:
chain = final_prompt | gpt

In [61]:
chain.invoke({"original_user_supplied_sentence": "DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland."})

Llama.generate: prefix-match hit

llama_print_timings:        load time =   10109.45 ms
llama_print_timings:      sample time =      42.36 ms /   178 runs   (    0.24 ms per token,  4201.78 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =   17148.11 ms /   178 runs   (   96.34 ms per token,    10.38 tokens per second)
llama_print_timings:       total time =   17681.93 ms /   178 tokens


{'id': 'chatcmpl-70ef76e0-b25a-434e-9cbd-ef779766b1c4',
 'object': 'chat.completion',
 'created': 1715524360,
 'model': 'C:/Users/benjc/Documents/models/Phi-3-mini-4k-instruct-q4.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '[\'DBA Pierre, who won the prestigious Booker Prize for his novel "Vernon God Little" and is now residing in Ireland\', \'Pierre DBA, celebrated with a Booker Prize win for "Vernon God Little", has recently made Ireland his home\', \'With fame from winning the Booker Prize for "Vernon God Little," Pierre DBC has decided to move to Ireland.\', \'"Vernon God Little" author DBC Pierre now lives in Ireland, continuing his literary journey after receiving a Booker Prize accolade.\' ]\n\nPlease note that while \'DBA\' is not standardly used as an abbreviation for names, it was creatively included here due to the constraint of using only capital letters and avoiding proper nouns. The correct representation would normally use the full

In [62]:
def paraphrase_sentences(sentence, n_runs, chain=chain):
    
    # Initialise the sentence list
    sentences = [sentence]
    new_sentence_amount = 1
    
    for i in range(1, n_runs + 1):
        
        print(f'  Iteration: {i}')
        # Break if no new sentences added in the previous loop
        if new_sentence_amount == 0:
            break
            
        while True:
            try:
                response = chain.invoke({"original_user_supplied_sentence":sentence})
                output_list = response['choices'][0]['message']['content']
                print(output_list)
                break
            except:
                pass
        
        new_sentence_amount = 0
        
        for output in output_list:
        
            if output not in sentences:
                sentences.append(output)
                new_sentence_amount += 1
            
    return sentences  

In [43]:
results = paraphrase_sentences("DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.", n_runs=10)

  Iteration: 1


Llama.generate: prefix-match hit

llama_print_timings:        load time =    9196.91 ms
llama_print_timings:      sample time =      25.91 ms /   115 runs   (    0.23 ms per token,  4439.30 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =   10433.84 ms /   115 runs   (   90.73 ms per token,    11.02 tokens per second)
llama_print_timings:       total time =   10758.56 ms /   115 tokens
Llama.generate: prefix-match hit


['DB Pierre, who won the prestigious Booker Prize for his novel "Vernon God Little," recently relocated to Ireland.', 'Renowned Booker Prize winner DB Pierre, known for his work on "Vernon God Little," has made a move to Ireland.', '"Vernon God Little" author DB Pierre, a recipient of the Booker Prize, is now living in Ireland.', 'DB Pierre, acclaimed for winning the Booker Prize with "Vernon God Little," has settled in Ireland.']
  Iteration: 2



llama_print_timings:        load time =    9196.91 ms
llama_print_timings:      sample time =      31.58 ms /   138 runs   (    0.23 ms per token,  4369.99 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =   12443.33 ms /   138 runs   (   90.17 ms per token,    11.09 tokens per second)
llama_print_timings:       total time =   12837.01 ms /   138 tokens
Llama.generate: prefix-match hit


['Pierre DBC, an acclaimed winner of the prestigious Booker Prize for his novel "Vernon God Little," has recently relocated to Ireland.', '"Vernon God Little" by Pierre DBC B., a celebrated recipient of the Booker Prize, is now being read in Ireland following his move there.', 'The renowned author Pierre DBC B., who won the Booker Prize for "Vernon God Little," has made Ireland his new home.']
<|assistant|> Pierre DBC, an accomplished writer and winner of the esteemed Booker Prize for his literary work "Vernon God Little," has recently settled in Ireland.
  Iteration: 3


...

['DBB Pierre, an acclaimed writer who won the prestigious Booker Prize for his novel "Vernon God Little", recently relocated to Ireland.', 'Known as DB Pierre, this celebrated author was awarded the Booker Prize and now resides in Ireland after completing a bestselling book titled "Vernon God Little".', 'As an esteemed recipient of the Booker Prize for his work on "Vernon God Little", DBC Pierre has made the move to Irish shores.']
  Iteration: 4



llama_print_timings:        load time =    9196.91 ms
llama_print_timings:      sample time =      32.61 ms /   133 runs   (    0.25 ms per token,  4079.13 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =   13738.30 ms /   133 runs   (  103.30 ms per token,     9.68 tokens per second)
llama_print_timings:       total time =   14150.98 ms /   133 tokens
Llama.generate: prefix-match hit


['DB Pierre, acclaimed recipient of the Booker Prize and the creative mind behind 'Vernon God Little,' recently relocated to Ireland.', "Famed for his Booker Prize win and the novel 'Vernon God Little,' DB Pierre has made a move to Ireland. His transition reflects an exciting new chapter in his life.", "'DB Pierre, known for winning the prestigious Booker Prize and writing Vernon God Little, now resides in Ireland.', "The esteemed author DB Pierre, who won the Booker Prize with 'Vernon God Little,' has recently settled down in Ireland.']
  Iteration: 5



llama_print_timings:        load time =    9196.91 ms
llama_print_timings:      sample time =      22.49 ms /    93 runs   (    0.24 ms per token,  4135.54 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =    9035.94 ms /    93 runs   (   97.16 ms per token,    10.29 tokens per second)
llama_print_timings:       total time =    9317.36 ms /    93 tokens


['Pierre DBC, the recipient of the prestigious Booker Prize for his novel "Vernon God Little," recently relocated to Ireland.', '"Vernon God Little" author Pierre DBC, who was honored with a Booker Prize, has made Ireland his new home.', 'Renowned Booker Prize winner and writer of "Vernon God Little," Pierre DBC, has now settled in Ireland.']
  Iteration: 6


In [45]:
results

['DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.',
 '[',
 "'",
 'D',
 'B',
 ' ',
 'P',
 'i',
 'e',
 'r',
 ',',
 'w',
 'h',
 'o',
 'n',
 't',
 'p',
 's',
 'g',
 'u',
 'k',
 'z',
 'f',
 'v',
 'l',
 '"',
 'V',
 'G',
 'd',
 'L',
 'c',
 'y',
 'a',
 'I',
 '.',
 'R',
 'm',
 ']',
 'C',
 'b',
 'T',
 '\n',
 '<',
 '|',
 '>',
 'K',
 'A',
 'F',
 'H',
 'x']